In [ ]:
import pandas as pd
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta, Variable, log, exp, bioDraws, bioMultSum, MonteCarlo, bioNormalCdf, Elem, RandomVariable, Integrate, PanelLikelihoodTrajectory, Derive
import biogeme.database as db
import numpy as np
import biogeme.distributions as dist
import biogeme.exceptions as excep
import biogeme.results as res
import biogeme.loglikelihood as ll
from biogeme.results import bioResults

In [ ]:
df = pd.read_csv('biogeme_data.csv')

In [ ]:
df['lowtreat'] = df['treatment'].replace({0: 1, 1: 0, 2: 0})
df['hightreat'] = df['treatment'].replace({0: 0, 1: 0, 2: 1})
df['chargavail_fraction'] = df['ev_chargavail'] / 100

In [ ]:
df = df.drop('race', axis=1)

In [ ]:
database = db.Database('df', df)

# mixed logit model, panel data
database.panel('resp_index')
globals().update(database.variables)

In [ ]:
# coefficients to be estimated for structural model
PR_intercept = Beta('PR_Intercept', 0, None, None, 0)
PR_lowtreat = Beta('PR_lowtreat', -0, None, None, 0)
PR_hightreat = Beta('PR_hightreat', 0, None, None, 0)

In [ ]:
# error component for structural model
sigma_s = Beta('sigma_s', 0.1, None, None, 0)
ec = sigma_s * bioDraws('ec', 'NORMAL')

In [ ]:
# define the structural model
PR = (
    PR_intercept
    + PR_hightreat*hightreat
    + PR_lowtreat*lowtreat
    + ec
)

In [ ]:
# measurement model

# initialize the coefficients to be estimated
INTER_Reliab02 = Beta('INTER_Reliab02', 0, None, None, 1)
INTER_Reliab03 = Beta('INTER_Reliab03', -0.5, None, None, 0)
INTER_Reliab04 = Beta('INTER_Reliab04', -0.5, None, None, 0)
INTER_Reliab05 = Beta('INTER_Reliab05', -0.5, None, None, 0)

B_Reliab02_F1 = Beta('B_Reliab02_F1', 1, None, None, 1)
B_Reliab03_F1 = Beta('B_Reliab03_F1', 1, None, None, 0)
B_Reliab04_F1 = Beta('B_Reliab04_F1',1, None, None, 0)
B_Reliab05_F1 = Beta('B_Reliab05_F1', 1, None, None, 0)

MODEL_Reliab02 = INTER_Reliab02 + B_Reliab02_F1 * PR
MODEL_Reliab03 = INTER_Reliab03 + B_Reliab03_F1 * PR
MODEL_Reliab04 = INTER_Reliab04 + B_Reliab04_F1 * PR
MODEL_Reliab05 = INTER_Reliab05 + B_Reliab05_F1 * PR

SIGMA_STAR_Reliab01 = Beta('SIGMA_STAR_Reliab01', 1, 1.0e-5, None, 1)
SIGMA_STAR_Reliab02 = Beta('SIGMA_STAR_Reliab02', 1, 1.0e-5, None, 1)
SIGMA_STAR_Reliab03 = Beta('SIGMA_STAR_Reliab03', 1, 1.0e-5, None, 0)
SIGMA_STAR_Reliab04 = Beta('SIGMA_STAR_Reliab04', 1, 1.0e-5, None, 0)
SIGMA_STAR_Reliab05 = Beta('SIGMA_STAR_Reliab05', 1, 1.0e-5, None, 0)

delta_1 = Beta('delta_1',1.5,0,10,0)
delta_2 = Beta('delta_2',1.5,0,10,0)
tau_1 = -delta_1 - delta_2
tau_2 = -delta_1 
tau_3 = 0
tau_4 = delta_1
tau_5 = delta_1 + delta_2

# define the indicator thresholds
Reliab02_tau_1 = (tau_1-MODEL_Reliab02) / SIGMA_STAR_Reliab02
Reliab02_tau_2 = (tau_2-MODEL_Reliab02) / SIGMA_STAR_Reliab02
Reliab02_tau_3 = (tau_3-MODEL_Reliab02) / SIGMA_STAR_Reliab02
Reliab02_tau_4 = (tau_4-MODEL_Reliab02) / SIGMA_STAR_Reliab02
Reliab02_tau_5 = (tau_5-MODEL_Reliab02) / SIGMA_STAR_Reliab02

Reliab03_tau_1 = (tau_1-MODEL_Reliab03) / SIGMA_STAR_Reliab03
Reliab03_tau_2 = (tau_2-MODEL_Reliab03) / SIGMA_STAR_Reliab03
Reliab03_tau_3 = (tau_3-MODEL_Reliab03) / SIGMA_STAR_Reliab03
Reliab03_tau_4 = (tau_4-MODEL_Reliab03) / SIGMA_STAR_Reliab03
Reliab03_tau_5 = (tau_5-MODEL_Reliab03) / SIGMA_STAR_Reliab03

Reliab04_tau_1 = (tau_1-MODEL_Reliab04) / SIGMA_STAR_Reliab04
Reliab04_tau_2 = (tau_2-MODEL_Reliab04) / SIGMA_STAR_Reliab04
Reliab04_tau_3 = (tau_3-MODEL_Reliab04) / SIGMA_STAR_Reliab04
Reliab04_tau_4 = (tau_4-MODEL_Reliab04) / SIGMA_STAR_Reliab04
Reliab04_tau_5 = (tau_5-MODEL_Reliab04) / SIGMA_STAR_Reliab04

Reliab05_tau_1 = (tau_1-MODEL_Reliab05) / SIGMA_STAR_Reliab05
Reliab05_tau_2 = (tau_2-MODEL_Reliab05) / SIGMA_STAR_Reliab05
Reliab05_tau_3 = (tau_3-MODEL_Reliab05) / SIGMA_STAR_Reliab05
Reliab05_tau_4 = (tau_4-MODEL_Reliab05) / SIGMA_STAR_Reliab05
Reliab05_tau_5 = (tau_5-MODEL_Reliab05) / SIGMA_STAR_Reliab05

# probit models for indicators
IndReliab02 = {
    1: bioNormalCdf(Reliab02_tau_1),
    2: bioNormalCdf(Reliab02_tau_2)-bioNormalCdf(Reliab02_tau_1),
    3: bioNormalCdf(Reliab02_tau_3)-bioNormalCdf(Reliab02_tau_2),
    4: bioNormalCdf(Reliab02_tau_4)-bioNormalCdf(Reliab02_tau_3),
    5: bioNormalCdf(Reliab02_tau_5)-bioNormalCdf(Reliab02_tau_4),
    6: 1-bioNormalCdf(Reliab02_tau_5),
    9999: 1.0
}

P_Reliab02 = Elem(IndReliab02, reliab_2)

IndReliab03 = {
    1: bioNormalCdf(Reliab03_tau_1),
    2: bioNormalCdf(Reliab03_tau_2)-bioNormalCdf(Reliab03_tau_1),
    3: bioNormalCdf(Reliab03_tau_3)-bioNormalCdf(Reliab03_tau_2),
    4: bioNormalCdf(Reliab03_tau_4)-bioNormalCdf(Reliab03_tau_3),
    5: bioNormalCdf(Reliab03_tau_5)-bioNormalCdf(Reliab03_tau_4),
    6: 1-bioNormalCdf(Reliab03_tau_5),
    9999: 1.0
}

P_Reliab03 = Elem(IndReliab03, reliab_3)

IndReliab04 = {
    1: bioNormalCdf(Reliab04_tau_1),
    2: bioNormalCdf(Reliab04_tau_2)-bioNormalCdf(Reliab04_tau_1),
    3: bioNormalCdf(Reliab04_tau_3)-bioNormalCdf(Reliab04_tau_2),
    4: bioNormalCdf(Reliab04_tau_4)-bioNormalCdf(Reliab04_tau_3),
    5: bioNormalCdf(Reliab04_tau_5)-bioNormalCdf(Reliab04_tau_4),
    6: 1-bioNormalCdf(Reliab04_tau_5),
    9999: 1.0
}

P_Reliab04 = Elem(IndReliab04, reliab_4)

IndReliab05 = {
    1: bioNormalCdf(Reliab05_tau_1),
    2: bioNormalCdf(Reliab05_tau_2)-bioNormalCdf(Reliab05_tau_1),
    3: bioNormalCdf(Reliab05_tau_3)-bioNormalCdf(Reliab05_tau_2),
    4: bioNormalCdf(Reliab05_tau_4)-bioNormalCdf(Reliab05_tau_3),
    5: bioNormalCdf(Reliab05_tau_5)-bioNormalCdf(Reliab05_tau_4),
    6: 1-bioNormalCdf(Reliab05_tau_5),
    9999: 1.0
}

P_Reliab05 = Elem(IndReliab05, reliab_5)

In [ ]:
# initialize coefficients for binary logit choice model
ASC_EV_1 = Beta('ASC_EV_1', 0, None, None, 0)
ASC_EV_1_NoHome = Beta('ASC_EV_1_NoHome', 0, None, None, 0)
B_PRICE_EV_1 = Beta('B_PRICE_EV_1', 0, None, None, 0)
B_PRICE_CONV_1 = Beta('B_PRICE_CONV_1', 0, None, None, 0)
B_OPCOST_EV_1 = Beta('B_OPCOST_EV_1', 0, None, None, 0)
B_OPCOST_CONV_1 = Beta('B_OPCOST_CONV_1', 0, None, None, 0)
B_RANGE_EV_1 = Beta('B_RANGE_EV_1', 0, None, None, 0)
B_RANGE_CONV_1 = Beta('B_RANGE_CONV_1', 0, None, None, 0)
B_CHARGAVAIL_EV_1_HOME = Beta('B_CHARGAVAIL_EV_1_HOME', 0, None, None, 0)
B_CHARGAVAIL_EV_1_NOHOME = Beta('B_CHARGAVAIL_EV_1_NOHOME', 0, None, None, 0)
B_CHARGAVAIL_EV_1 = Beta('B_CHARGAVAIL_EV_1', 0, None, None, 0)
B_PR_1_NOHOME = Beta('B_PR_1_NOHOME', 0, None, None, 0)
B_PR_1 = Beta('B_PR_1', 0, None, None, 0)




In [ ]:
# define the utility functions
V_EV_1 = (
    ASC_EV_1
    + ASC_EV_1_NoHome*(1-homecharge)
    + B_PRICE_EV_1* (ev_price/budget)
    + B_OPCOST_EV_1*ev_opcost
    + B_RANGE_EV_1*(ev_range/100)
    + B_CHARGAVAIL_EV_1*chargavail_fraction
    + B_CHARGAVAIL_EV_1_NOHOME*chargavail_fraction*(1-homecharge)
    + B_PR_1*PR
    + B_PR_1_NOHOME*PR*(1-homecharge)
)

V_CONV_1 = (
    B_PRICE_CONV_1* (icev_price/budget)
    + B_OPCOST_CONV_1*icev_opcost
    + B_RANGE_CONV_1*(icev_range/100)
)


In [ ]:
# link utility and availability

V1 = {1: V_EV_1, 2: V_CONV_1}
av = {1: ev_av, 2: icev_av}

In [ ]:
# define the probability functions
prob1 = models.logit(V1, av, choice)

# link the measurement model
condlike = prob1 * P_Reliab03 * P_Reliab04 * P_Reliab05 * P_Reliab02

# set panel structure
condlikeindiv = PanelLikelihoodTrajectory(condlike)
loglike = log(MonteCarlo(condlike))


In [ ]:
# define the model
the_biogeme = bio.BIOGEME(database, loglike,numberOfDraws = 1000)
the_biogeme.modelName = 'PR_ICLV_Mixed_MonteCarlo'

In [ ]:
# run model and print results
model_results = the_biogeme.estimate()
pandasResults = model_results.getEstimatedParameters()
print(pandasResults)